In [1]:
from dotenv import load_dotenv

load_dotenv()
from mlip_arena.tasks.thermal_conductivity.input import save_to_db

from ase.io import read

atoms_list = read("./phononDB-PBE-structures.extxyz", index=":", format="extxyz")

save_to_db(atoms_list, "phononDB-PBE.db")

2024-12-22 17:55:44.898 | INFO     | mlip_arena.tasks.thermal_conductivity:<module>:6 - 
The module 'mlip_arena.tasks.thermal_conductivity' is adapted from the repository: https://github.com/MPA2suite/k_SRME. 
By using this module, you agree to the terms and conditions specified in the following license: 

https://github.com/MPA2suite/k_SRME/blob/main/LICENSE

Additionally, please ensure proper attribution by citing the following reference: 

- Póta, B., Ahlawat, P., Csányi, G., & Simoncelli, M. (2024). Thermal Conductivity Predictions with Foundation Atomistic Models. arXiv preprint arXiv:2408.00755.

A local copy of the LICENSE file can be found at: /pscratch/sd/c/cyrusyc/mlip-arena/mlip_arena/tasks/thermal_conductivity/LICENSE.



No module named 'deepmd'


2024-12-22 17:55:47.369 | INFO     | mlip_arena.tasks.thermal_conductivity.input:save_to_db:45 - phononDB-PBE.db uploaded to atomind/mlip-arena/thermal_conductivity


PosixPath('phononDB-PBE.db')

## Run thermal conductivity task locally

In [3]:
from mlip_arena.tasks.thermal_conductivity.task import get_thermal_conductivity
from mlip_arena.tasks.thermal_conductivity.input import get_atoms_from_extxyz
from mlip_arena.models import MLIPEnum

atoms = next(get_atoms_from_extxyz("phononDB-PBE-structures.extxyz"))

for model in MLIPEnum:
    result = get_thermal_conductivity.fn(
        atoms=atoms,
        calculator_name=model.name
    )
    break

22:22:56.492 | INFO    | Task run 'get_atoms_from_extxyz' - Created task run 'get_atoms_from_extxyz' for task 'get_atoms_from_extxyz'

22:22:56.529 | INFO    | Task run 'get_atoms_from_extxyz' - Finished in state Completed()

Selected GPU cuda:0 with 40194.44 MB free memory from 1 GPUs
Default dtype float32 does not match model dtype float64, converting models to float32.
2024-12-22 22:22:56 - Symmetry: prec 1e-05 got symmetry group number 225 , international (Hermann-Mauguin) Fm-3m , Hall  -F 4 2 3


22:22:56.939 | INFO    | Task run 'OPT: Pb4Te4 - MACE-MP(M)' - Created task run 'OPT: Pb4Te4 - MACE-MP(M)' for task 'OPT'

22:22:57.398 | INFO    | Task run 'OPT: Pb4Te4 - MACE-MP(M)' - Finished in state Cached(type=COMPLETED)

2024-12-22 22:22:57 - Symmetry: prec 1e-05 got symmetry group number 225 , international (Hermann-Mauguin) Fm-3m , Hall  -F 4 2 3


22:22:57.816 | INFO    | Task run 'OPT: Pb4Te4 - MACE-MP(M)' - Created task run 'OPT: Pb4Te4 - MACE-MP(M)' for task 'OPT'

Selected GPU cuda:0 with 40129.95 MB free memory from 1 GPUs
Using device: cuda:0
Selected GPU cuda:0 with 40129.95 MB free memory from 1 GPUs
Default dtype float32 does not match model dtype float64, converting models to float32.
Using calculator: <mlip_arena.models.externals.mace-mp.MACE_MP_Medium object at 0x7f59344f7990>
Using filter: <ase.filters.FrechetCellFilter object at 0x7f59cc17e8d0>
Using optimizer: <ase.optimize.fire.FIRE object at 0x7f59cc4b3e10>
      Step     Time          Energy          fmax
FIRE:    0 22:22:58      -30.626575        0.000000


22:22:58.317 | INFO    | Task run 'OPT: Pb4Te4 - MACE-MP(M)' - Finished in state Completed()

2024-12-22 22:22:58 - Symmetry: prec 1e-05 got symmetry group number 225 , international (Hermann-Mauguin) Fm-3m , Hall  -F 4 2 3


In [4]:
result

{'force': {'fc2_set': array([[[-5.37725165e-02, -1.98128669e-09, -1.09896064e-07],
          [ 1.30096143e-02,  2.37876330e-10,  0.00000000e+00],
          [-7.03117257e-05, -2.31597874e-09, -1.86264515e-09],
          ...,
          [ 2.67593423e-05,  3.38032842e-05,  3.80352139e-06],
          [-4.57148781e-05,  1.17830932e-05, -3.76254320e-07],
          [-5.19048845e-05, -9.46447253e-05, -6.86142594e-05]],
  
         [[ 1.13546150e-04, -1.12909055e-03, -1.12909451e-03],
          [ 2.28802790e-04,  9.35941120e-04,  9.35951248e-04],
          [-1.60758485e-04,  5.72525751e-05,  5.72502613e-05],
          ...,
          [-1.27753742e-06, -3.78973782e-05, -1.15111470e-06],
          [-3.55979341e-06,  3.13669443e-06,  4.61004674e-06],
          [ 4.16665458e-09, -1.86264515e-09,  3.72529030e-09]]],
        dtype=float32),
  'fc3_set': array([[[-5.38115650e-02,  1.42066137e-09,  1.86264515e-09],
          [ 2.57739089e-02, -1.68573455e-09, -1.86264515e-09],
          [ 1.42478631e-04,